In [2]:
from transformers import pipeline

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [4]:
nlp = spacy.load("en_core_web_lg")

In [43]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')
a = data_train.size
b = data_test.size
total = a+b

description_train = data_train['description']
description_test = data_test['description']
cvss_train = data_train['base_score']
cvss_test = data_test['base_score']

print(description_train[1])
print(data_train.size)
print(data_test.size)
print(description_train.size)
print(cvss_train.size)


An information disclosure vulnerability in the Android media framework (n/a). Product: Android. Versions: 7.0, 7.1.1, 7.1.2, 8.0. Android ID: A-63873837.
985856
246464
61616
61616


In [39]:
data = pd.concat([data_train, data_test], ignore_index=True)
desc = data['description']
cvss = data['base_score']

print(desc[0])
print(cvss.size)

A remote code exection vulnerability was identified in HPE Intelligent Management Center (IMC) PLAT earlier than version 7.3 E0506P09.
77020


In [19]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [34]:
print(desc[1])
print('Tokenized: ', tokenizer.tokenize(desc[1]))

An information disclosure vulnerability in the Android media framework (n/a). Product: Android. Versions: 7.0, 7.1.1, 7.1.2, 8.0. Android ID: A-63873837.
Tokenized:  ['an', 'information', 'disclosure', 'vulnerability', 'in', 'the', 'android', 'media', 'framework', '(', 'n', '/', 'a', ')', '.', 'product', ':', 'android', '.', 'versions', ':', '7', '.', '0', ',', '7', '.', '1', '.', '1', ',', '7', '.', '1', '.', '2', ',', '8', '.', '0', '.', 'android', 'id', ':', 'a', '-', '63', '##8', '##7', '##38', '##37', '.']


In [35]:
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(desc[1])))

Token IDs:  [2019, 2592, 19380, 18130, 1999, 1996, 11924, 2865, 7705, 1006, 1050, 1013, 1037, 1007, 1012, 4031, 1024, 11924, 1012, 4617, 1024, 1021, 1012, 1014, 1010, 1021, 1012, 1015, 1012, 1015, 1010, 1021, 1012, 1015, 1012, 1016, 1010, 1022, 1012, 1014, 1012, 11924, 8909, 1024, 1037, 1011, 6191, 2620, 2581, 22025, 24434, 1012]


In [36]:
#finding the length of the longest description
max_len = 0
counter = 0

desc_new = []
cvss_new = []

for description in range(len(desc)):
    if len(desc[description]) <= 512:
        desc_new.append(desc[description])
        cvss_new.append(cvss[description])

        #tokenize the text and add CLS and SEP tokens

        input_ids = tokenizer.encode(desc[description], add_special_tokens=True)

        # Update the maximum description length
        max_len = max(max_len, len(input_ids))

print(len(desc))
print(len(desc_new))
print('Max description length: ', max_len)

77020
67152
Max description length:  294


In [44]:
# Tokenize all descriptions and map the tokens to their word IDs
import torch
input_ids = []
attention_masks = []

for description in desc_new:
    encoded_dict = tokenizer.encode_plus(
                                description,
                                add_special_tokens = True,
                                max_length = 64,
                                pad_to_max_length = True,
                                return_attention_mask = True,
                                return_tensors = 'pt'    
                            )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

#convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(cvss_new)

print('Original: ', desc_new[1])
print('Token IDs: ', input_ids[1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Lenovo\miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  An information disclosure vulnerability in the Android media framework (n/a). Product: Android. Versions: 7.0, 7.1.1, 7.1.2, 8.0. Android ID: A-63873837.
Token IDs:  tensor([  101,  2019,  2592, 19380, 18130,  1999,  1996, 11924,  2865,  7705,
         1006,  1050,  1013,  1037,  1007,  1012,  4031,  1024, 11924,  1012,
         4617,  1024,  1021,  1012,  1014,  1010,  1021,  1012,  1015,  1012,
         1015,  1010,  1021,  1012,  1015,  1012,  1016,  1010,  1022,  1012,
         1014,  1012, 11924,  8909,  1024,  1037,  1011,  6191,  2620,  2581,
        22025, 24434,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [45]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, torch.Tensor(cvss_new))

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

53,721 training samples
13,431 validation samples


In [46]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [48]:
import torch.nn as nn
from transformers import BertModel
class BertRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_bert=False):
        
        super(BertRegressor, self).__init__()
        D_in, D_out = 768, 1
        
        self.bert = \
                   BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):
        
        outputs = self.bert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs
model = BertRegressor(drop_rate=0.2)

Downloading: 100%|██████████| 420M/420M [00:59<00:00, 7.45MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

No GPU available, using the CPU instead.


BertRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [49]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

In [50]:
from transformers import get_linear_schedule_with_warmup
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)

In [51]:
loss_function = nn.MSELoss()

In [55]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader): 
            print(step)  
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)           
            loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
                
    return model
model = train(model, optimizer, scheduler, loss_function, epochs, 
              train_dataloader, device, clip_value=2)

0
-----
0


C:\Users\Lenovo\AppData\Local\Temp/ipykernel_18688/1718634571.py:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), clip_value)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138


KeyboardInterrupt: 

In [ ]:
def evaluate(model, loss_function, test_dataloader, device):
    model.eval()
    test_loss, test_r2 = [], []
    for batch in test_dataloader:
        batch_inputs, batch_masks, batch_labels = \
                                 tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs, batch_labels)
        test_loss.append(loss.item())
        r2 = r2_score(outputs, batch_labels)
        test_r2.append(r2.item())
    return test_loss, test_r2
def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, _ = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, 
                            batch_masks).view(1,-1).tolist()[0]
    return output